In [1]:
import os
import numpy as np
import scanpy as sc
import time
from pathlib import Path
import torch
import concord as ccd
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
import matplotlib as mpl

from matplotlib import font_manager, rcParams
custom_rc = {
    'font.family': 'Arial',  # Set the desired font for this plot
}

mpl.rcParams['svg.fonttype'] = 'none'
mpl.rcParams['pdf.fonttype'] = 42

/wynton/group/gartner/zuzhi/.conda/envs/scenv/lib/python3.13/site-packages/louvain/__init__.py:54: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import get_distribution, DistributionNotFound


In [ ]:
proj_name = "pbmc_DeSimone"
save_dir = f"../save/{proj_name}-{time.strftime('%b%d')}/"
save_dir = Path(save_dir)
save_dir.mkdir(parents=True, exist_ok=True)

data_dir = f"../data/{proj_name}/"
data_dir = Path(data_dir)
data_dir.mkdir(parents=True, exist_ok=True)
device = torch.device('cuda:3' if torch.cuda.is_available() else 'cpu')
print(device)
seed = 0
ccd.ul.set_seed(seed)

file_suffix = f"{time.strftime('%b%d-%H%M')}"
file_suffix

cuda:3


'Jun12-1635'

In [3]:
ANNDATA_OBJECT_DIR='../data/pbmc_Darmanis/'
adata = sc.read_h5ad(ANNDATA_OBJECT_DIR + 'pbmc_Darmanis_subset_9K.h5ad')

In [4]:
batch_key = 'dataset'
state_key = 'cell_type'

In [5]:
combined_keys = [
        # "unintegrated",
        # "scanorama", "liger", "harmony",
        # "scvi", "scanvi",
        "concord",
        # "concord_class", 
        # "concord_decoder", "contrastive"
    ]

In [7]:
time_log, ram_log, vram_log = ccd.ul.run_integration_methods_pipeline(
    adata=adata,                          # Your input AnnData object
    methods=combined_keys,            # List of methods to run
    batch_key=batch_key,                    # Column in adata.obs for batch info
    count_layer="counts",                 # Layer name containing raw counts
    class_key=state_key,               # Column in adata.obs for class labels (used in SCANVI and CONCORD variants)
    latent_dim=30,                        # Latent dimensionality for PCA and embeddings
    device=device,                        # Or "cpu", or "mps" for Apple Silicon
    return_corrected=False,                   # Whether to store corrected expression matrices
    transform_batch=None,                 # Optionally specify a batch to transform to in scVI
    seed=42,                              # Random seed for reproducibility
    compute_umap=False,                    # Run UMAP for all output embeddings
    umap_n_components=2,
    umap_n_neighbors=30,
    umap_min_dist=0.5,
    verbose=True,                        # Print progress messages
)

Concord - WARNING - save_dir is None. Model and log files will not be saved.
Concord - WARNING - No input feature list provided. It is recommended to first select features using the command `concord.ul.select_features()`.


FAISS not found. Using sklearn for k-NN computation.


p_intra_knn: 0.3


Epoch 0 Training: 119it [00:02, 45.21it/s, loss=3.88] 
Epoch 9 Training: 100%|██████████| 119/119 [00:01<00:00, 108.00it/s, loss=3.65]

Concord - WARNING - save_dir is None. Skipping model/config saving.



concord completed in 14.12 sec.
✅ Selected methods completed.


In [8]:
adata

AnnData object with n_obs × n_vars = 8500 × 2000
    obs: 'kit', 'organism_ontology_term_id', 'tissue_ontology_term_id', 'tissue_type', 'assay_ontology_term_id', 'disease_ontology_term_id', 'cell_type_ontology_term_id', 'self_reported_ethnicity_ontology_term_id', 'development_stage_ontology_term_id', 'sex_ontology_term_id', 'donor_id', 'suspension_type', 'predicted_celltype', 'is_primary_data', 'cell_type', 'assay', 'disease', 'organism', 'sex', 'tissue', 'self_reported_ethnicity', 'development_stage', 'observation_joinid', 'dataset'
    var: 'feature_is_filtered', 'feature_reference', 'feature_biotype', 'gene_name-10X_3-rep1_annotated', 'highly_deviant-10X_3-rep1_annotated', 'feature_name-10X_3-rep1_annotated', 'feature_length-10X_3-rep1_annotated', 'feature_type-10X_3-rep1_annotated', 'gene_name-10X_3-rep2_annotated', 'highly_deviant-10X_3-rep2_annotated', 'feature_name-10X_3-rep2_annotated', 'feature_length-10X_3-rep2_annotated', 'feature_type-10X_3-rep2_annotated', 'gene_name-10X_5